# Mybatis
* MyBatis is a first class persistence framework with support for custom SQL, stored procedures and advanced mappings. MyBatis eliminates almost all of the JDBC code and manual setting of parameters and retrieval of results. MyBatis can use simple XML or Annotations for configuration and map primitives, Map interfaces and Java POJOs (Plain Old Java Objects) to database records.
* SqlSessionFactoryBuilder: 用于创建SqlSessionFactory
    * 这个类可以被实例化、使用和丢弃，一旦创建了 SqlSessionFactory，就不再需要它了。因此 SqlSessionFactoryBuilder 实例的最佳作用域是方法作用域（也就是局部方法变量）。你可以重用 SqlSessionFactoryBuilder 来创建多个 SqlSessionFactory 实例，但是最好还是不要让其一直存在以保证所有的 XML 解析资源开放给更重要的事情
* SqlSessionFactory
    * SqlSessionFactory 一旦被创建就应该在应用的运行期间一直存在，没有任何理由对它进行清除或重建。使用 SqlSessionFactory 的最佳实践是在应用运行期间不要重复创建多次，多次重建 SqlSessionFactory 被视为一种代码“坏味道（bad smell）”。因此 SqlSessionFactory 的最佳作用域是应用作用域。有很多方法可以做到，最简单的就是使用单例模式或者静态单例模式
    * 每个基于 MyBatis 的应用都是以一个 SqlSessionFactory 的实例为中心的。SqlSessionFactory 的实例可以通过 SqlSessionFactoryBuilder 获得。而 SqlSessionFactoryBuilder 则可以从 XML 配置文件或一个预先定制的 Configuration 的实例构建出 SqlSessionFactory 的实例
    * 
    ```java
        DataSource dataSource = BlogDataSourceFactory.getBlogDataSource();
        TransactionFactory transactionFactory = new JdbcTransactionFactory();
        Environment environment = new Environment("development", transactionFactory, dataSource);
        Configuration configuration = new Configuration(environment);
        configuration.addMapper(BlogMapper.class);
        SqlSessionFactory sqlSessionFactory = new SqlSessionFactoryBuilder().build(configuration);
    ```
* SqlSession
    * 每个线程都应该有它自己的 SqlSession 实例。SqlSession 的实例不是线程安全的，因此是不能被共享的，所以它的最佳的作用域是请求或方法作用域。绝对不能将 SqlSession 实例的引用放在一个类的静态域，甚至一个类的实例变量也不行。也绝不能将 SqlSession 实例的引用放在任何类型的管理作用域中，比如 Servlet 架构中的 HttpSession。如果你现在正在使用一种 Web 框架，要考虑 SqlSession 放在一个和 HTTP 请求对象相似的作用域中。换句话说，每次收到的 HTTP 请求，就可以打开一个 SqlSession，返回一个响应，就关闭它。这个关闭操作是很重要的，你应该把这个关闭操作放到 finally 块中以确保每次都能执行关闭
    * SqlSession 完全包含了面向数据库执行 SQL 命令所需的所有方法。你可以通过 SqlSession 实例来直接执行已映射的 SQL 语句
    *
    ```java
        SqlSession session = sqlSessionFactory.openSession();
        try {
          Blog blog = (Blog) session.selectOne("org.mybatis.example.BlogMapper.selectBlog", 101);
        } finally {
          session.close();
        }
    ```
    * 
    ```java
        SqlSession session = sqlSessionFactory.openSession();
        try {
          BlogMapper mapper = session.getMapper(BlogMapper.class);
          Blog blog = mapper.selectBlog(101);
        } finally {
          session.close();
        }
    ```
* Mapper: 定义方法与SQL的映射接口，可以基于XML定义Mapper，也可以基于Java注解的方式定义Mapper
    * 映射器是一个你创建来绑定你映射的语句的接口。映射器接口的实例是从 SqlSession 中获得的。因此从技术层面讲，任何映射器实例的最大作用域是和请求它们的 SqlSession 相同的。尽管如此，映射器实例的最佳作用域是方法作用域。也就是说，映射器实例应该在调用它们的方法中被请求，用过之后即可废弃。并不需要显式地关闭映射器实例，尽管在整个请求作用域（request scope）保持映射器实例也不会有什么问题，但是很快你会发现，像 SqlSession 一样，在这个作用域上管理太多的资源的话会难于控制。所以要保持简单，最好把映射器放在方法作用域（method scope）内
* 配置参考(mybatis.config.xml): http://www.mybatis.org/mybatis-3/zh/configuration.html
* 基于XML的sql映射配置参考(xxxMapper.xml): http://www.mybatis.org/mybatis-3/zh/sqlmap-xml.html
* 基于JAVA注解的sql映射配置参考(xxxMapper.java): http://www.mybatis.org/mybatis-3/zh/java-api.html
* 动态sql配置: http://www.mybatis.org/mybatis-3/zh/dynamic-sql.html
* 简化版querydsl功能-SQL语句构建器类：http://www.mybatis.org/mybatis-3/zh/statement-builders.html

****

# Hibernate
* https://docs.jboss.org/hibernate/orm/5.4/userguide/html_single/Hibernate_User_Guide.html
* Hibernate takes care of the mapping from Java classes to database tables, and from Java data types to SQL data types. In addition, it provides data query and retrieval facilities. It can significantly reduce development time otherwise spent with manual data handling in SQL and JDBC. Hibernate’s design goal is to relieve the developer from 95% of common data persistence-related programming tasks by eliminating the need for manual, hand-crafted data processing using SQL and JDBC. 

****

### 架构
* 层次结构:
<img src="../images/hibernate/data_access_layers.svg" width="200px"/>
* 类之间的关系
<img src="../images/hibernate/JPA_Hibernate.svg" width="500px">

****

### 类
* StandardServiceRegistry
    * incorporates configuration information into a working set of Services for use by the SessionFactory, loading `hibernate.cfg.xml`
* MetadataSources
    * the start point for telling Hibernate about your domain model
* Metadata
    * represents the complete, partially validated view of the application domain model which the SessionFactory will be based on
* SessionFactory
    * A thread-safe (and immutable) representation of the mapping of the application domain model to a database. Acts as a factory for org.hibernate.Session instances. The EntityManagerFactory is the JPA equivalent of a SessionFactory and basically, those two converge into the same SessionFactory implementation.
* Session
    *  A single-threaded, short-lived object conceptually modeling a "Unit of Work" PoEAA. In JPA nomenclature, the Session is represented by an EntityManager.
    * Behind the scenes, the Hibernate Session wraps a JDBC java.sql.Connection and acts as a factory for org.hibernate.Transaction instances. It maintains a generally "repeatable read" persistence context (first level cache) of the application domain model.
    * 
    ```java
        Session session = sessionFactory.openSession();
        session.beginTransaction();
        session.save( new Event( "Our very first event!", new Date() ) );
        session.save( new Event( "A follow up event", new Date() ) );
        session.getTransaction().commit();
        session.close();
    ```
    * 
    ```java
        session = sessionFactory.openSession();
        session.beginTransaction();
        List result = session.createQuery( "from Event" ).list();
        for ( Event event : (List<Event>) result ) {
            System.out.println( "Event (" + event.getDate() + ") : " + event.getTitle() );
        }
        session.getTransaction().commit();
        session.close();
    ```
* Transaction
    * A single-threaded, short-lived object used by the application to demarcate individual physical transaction boundaries. EntityTransaction is the JPA equivalent and both act as an abstraction API to isolate the application from the underlying transaction system in use (JDBC or JTA).
    
****

### 应用
* Native Hibernate APIs and hbm.xml Mapping
    * `hibernate.cfg.xml` file defines the Hibernate configuration information
    * The `connection.driver_class, connection.url, connection.username and connection.password <property/>` elements define JDBC connection information
    * The `dialect` property specifies the particular SQL variant with which Hibernate will converse
    * The `hbm2ddl.auto` property enables automatic generation of database schemas directly into the database
    * The mapping file is the classpath `resource org/hibernate/tutorial/hbm/XXX.hbm.xml`
* Native Hibernate APIs and Annotation Mappings
    * `@javax.persistence.Entity, @javax.persistence.Table, @javax.persistence.Id, @javax.persistence.GeneratedValue`
    * `@org.hibernate.annotations.GenericGenerator`
* Java Persistence API (JPA)
    * JPA defines a different bootstrap process that uses its own configuration file named `persistence.xml`
    * `EntityManagerFactory`
    ```java
        sessionFactory = Persistence.createEntityManagerFactory( "org.hibernate.tutorial.jpa" );
        EntityManager entityManager = sessionFactory.createEntityManager();
        entityManager.getTransaction().begin();
        entityManager.persist( new Event( "Our very first event!", new Date() ) );
        entityManager.persist( new Event( "A follow up event", new Date() ) );
        entityManager.getTransaction().commit();
        entityManager.close();
    ```
* Naming strategies
    * Part of the mapping of an object model to the relational database is mapping names from the object model to the corresponding database names. Hibernate looks at this as 2 stage process:
        * The first stage is determining a proper logical name from the domain model mapping. A logical name can be either explicitly specified by the user (using @Column or @Table e.g.) or it can be implicitly determined by Hibernate through an ImplicitNamingStrategy contract.
        * Second is the resolving of this logical name to a physical name which is defined by the PhysicalNamingStrategy contract.
    * details reference to here: http://docs.jboss.org/hibernate/orm/5.4/userguide/html_single/Hibernate_User_Guide.html#naming
* 映射实战参考连接: http://docs.jboss.org/hibernate/orm/5.4/userguide/html_single/Hibernate_User_Guide.html#domain-model
* 配置参考: http://docs.jboss.org/hibernate/orm/5.4/userguide/html_single/Hibernate_User_Guide.html#configurations
* 注解参考: https://docs.jboss.org/hibernate/orm/5.4/userguide/html_single/Hibernate_User_Guide.html#annotations

****

# 关系映射
<table>
    <tr>
        <td colspan='4'>
            <h3><center>主表是将主键借出去的表，从表是拥有别的表主键的表，双向关系是实体类相互引用，单向关系是只有一方实体类引用</center></h3>
        </td>
    </tr>
    <tr>
        <td colspan='2'>
            <h4><center>无关联关系表的双向关系</center></h4>
        </td>
        <td colspan='2'>
            <h4><center>无关联关系表的单向关系</center></h4>
        </td>
    </tr>
    <tr>
        <td>
            <h4><center>双向一对一</center></h4>
            <b><center><code>@OneToOne(mappedBy="filed_name")</code>在主表实例类中</center></b>
            <img src='../images/hibernate/双向一对一.png' width='400px'>
            <i><p>实体Employee被映射为名字为EMPLOYEE的表；
                实体Cubicle被映射为名字为CUBICLE的表；
                表EMPLOYEE有一个指向CUBICLE的外键
            </p></i>
        </td>
        <td>
            <h4><center>双向多对一或一对多</center></h4>
            <b><center><code>@OneToMany(mappedBy="filed_name")</code>在主表实例类中</center></b>
            <img src='../images/hibernate/双向一对多.png' width='400px'>
            <i><p>实体Employee映射成表名为EMPLOYEE的表；
                实体Department映射成表明为DEPARTMENT的表；
                表EMPLOYEE有一个指向DEPARTMENT的外键
            </p></i>
        </td>
        <td>
            <h4><center>单向一对一</center></h4>
            <b><center><code>@OneToOne</code>在从表实例类中</center></b>
            <img src='../images/hibernate/单向一对一.png' width='400px'>
            <i><p>实体Employee映射为名称为EMPLOYEE的表；
                实体TravelProfile映射为名称为TRAVELPROFILE的表；
                表EMPLOYEE有一个指向TRAVELPROFILE的外键
            </p></i>
        </td>
        <td>
            <h4><center>单向多对一</center></h4>
            <b><center><code>@ManyToOne</code>在从表实例类中</center></b>
            <img src='../images/hibernate/单向多对一.png' width='400px'>
            <i><p>实体Employee映射成表EMPLOYEE；
                实体Address映射成表ADDRESS；
                表EMPLOYEE有一个指向表ADDRESS的外键
            </p></i>
        </td>
    </tr>
     <tr>
        <td colspan='2'>
            <h4><center>有关联关系表的单向关系</center></h4>
        </td>
         <td colspan='2'>
            <h4><center>有关联关系表的双向关系</center></h4>
        </td>
    </tr>
    <tr>
        <td>
            <h4><center>单向一对多</center></h4>
            <b><center><code>@OneToMany</code>在拥有方实例类中</center></b>
            <img src='../images/hibernate/单向一对多.png' width='400px'>
            <i><p>实体Employee引用一个实体AnnualReview的集合；
                实体AnnualReview没有引用Employee；
                有一个关联表EMPLOYEE_ANNUALREVIEW；
                关联表有两个外键分别是表AnnualReview和表EMPLOYEE的主键
            </p></i>
        </td>
        <td>
            <h4><center>单向多对多</center></h4>
            <b><center><code>@ManyToMany</code>在拥有方实例类中</center></b>
            <img src='../images/hibernate/单向多对多.png' width='400px'>
            <i><p>实体Employee映射成表EMPLOYEE；
                实体Patent映射成表PATENT；
                有一个关联表EMPLOYEE_PATENT；
                关联表有两个外键分别是表PATENT和表EMPLOYEE的主键
            </p></i>
        </td>
        <td colspan='2'>
            <h4><center>双向多对多</center></h4>
            <b><center><code>@ManyToMany(mappedBy="filed_name")</code>在拥有方实例类中</center></b>
            <img src='../images/hibernate/双向多对多.png' width='400px'>
            <i><p>实体Project映射成表PROJECT；
                实体Employee映射成表EMPLOYEE；
                有一个关联表PROJECT_EMPLOYEE；
                关联表有两个外键分别是表PROJECT和表EMPLOYEE的主键
            </p></i>
        </td>
    </tr>
    <tr>
</table>